In [39]:
import google.generativeai as genai
import json
import re
from tqdm import tqdm # 用於顯示進度條
import csv
import os

# --- 1. 常數與提示詞定義 (來自您的 GRPO 腳本) ---

SYSTEM_PROMPT = """你是一名專門根據患者用藥與病史紀錄預測 ICD-10 編碼的醫學專家。  
請嚴格遵守以下作答格式：  

<reasoning>  
逐步推理：逐條說明病例中每個臨床資訊（症狀、診斷、檢查結果、既往病史、用藥等）可能對應的 ICD-10 編碼，並在最後總結出最合理的編碼集合。  
</reasoning>  
<answer>  
僅輸出最終的 ICD-10 編碼，使用英文逗號分隔，不要添加任何額外文字或解釋。  
</answer>  
"""
EXAMPLE_QUESTION = """
Sex: F
Service: MEDICINE
Allergies: Aspirin
Attending: ___
Chief Complaint: weakness, diarrhea
Major Surgical or Invasive Procedure: None

History of Present Illness:
Ms. ___ is a ___ year-old woman with PMH significant for chronic anemia, osteoporosis, hypertension, ataxia, and recent L5 fracture in the setting of recurrent falls who presents from home with fatigue and generalized weakness and diarrhea.  

The patient's recent history is notable for the follow:  
- On ___, she presented with 4 days of LBP s/p fall from standing at which time imaging revealed acute L5 fracture. She was evaluated by Spine team who recommended early mobilization, pain control, but no brace required. She was evaluated by ___, and discharged to ___.  
- She was discharged home with ___ on ___.  
- On ___, she again presented to ___ s/p fall from standing while trying to reach for a glass of water. She did have a occipital scalp hematoma, but imaging including ___, C-spine CT, and L hip X-ray were negative for acute process so patient was discharged home.  

She now represents with generalized fatigue and diarrhea. In the setting of opiates for her L5 fracture, the patient has had constipation (5 days with no BM) for which she took a "natural laxative" the evening prior to presentation. The patient had 2 bowel movements in the morning of presentation and one episode of incontinence with diarrhea while sleeping. In this setting, she felt very weak and called EMS and was brought to ___ ED.

What ICD-10 codes should be assigned? Please just tell me which codes in the end
"""
EXAMPLE_REASONING = """The patient has chronic anemia, documented and clinically relevant, which corresponds to D500.  
There is vitamin deficiency suggested, which fits E538.  
She also has visual impairment history, which maps to H548.  
Hypertension is a chronic condition, coded as I10.  
Diarrhea due to laxative use is coded as K521 (toxic gastroenteritis and colitis).  
Her recent fracture with underlying osteoporosis corresponds to M810.  
Ataxia is a chronic neurologic condition, coded as R270.  
There is adverse effect of drugs (laxatives/opioids), so T474X5A applies.  
Environmental factor of injury is Y92099 (unspecified place of occurrence).  
She also has aspirin allergy, captured by Z9181.  

Therefore, the most appropriate ICD-10 codes are:
"""

EXAMPLE_ANSWER = "D500 E538 H548 I10 K521 M810 R270 T474X5A Y92099 Z9181"

In [40]:
def extract_icd_answer(text: str) -> str:
    """
    從模型的完整回覆或標準答案中提取 ICD-10 編碼。
    優先從 <answer> 標籤中提取,若無則從整個文本中提取。
    """
    # 使用正則表達式搜尋 <answer> 標籤，re.DOTALL 讓 . 可匹配換行符，re.IGNORECASE 忽略大小寫
    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL | re.IGNORECASE)
    # 如果找到 <answer> 標籤，則只在標籤內容中搜尋 ICD 編碼；否則在整個文本中搜尋
    search_text = answer_match.group(1) if answer_match else text
    
    # 匹配格式：字母 + 1-3位數字 + 可選的小數點和1-2位數字
    codes_list = re.findall(r'\b([A-Z]\d{1,6}(?:\.\d{1,2})?)\b', search_text, re.IGNORECASE)
    
    # 移除小數點、去重、排序、轉大寫，並過濾掉長度<=2的編碼
    cleaned_codes = sorted(set(
        code.replace('.', '').upper() 
        for code in codes_list 
        if len(code.replace('.', '')) > 2
    ))
    return " ".join(cleaned_codes)



def extract_xml_answer(text: str) -> str:
    """提取 <answer> 標籤內的內容"""
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    answer = answer.strip()
    return answer


def is_text(text: str) -> bool:
    """判斷輸入字串是否主要為文字敘述（而非 ICD code 等短編碼）"""
    letters_and_spaces = sum(c.isalpha() or c.isspace() for c in text)
    ratio = letters_and_spaces / max(1, len(text))  
    return ratio > 0.5


def compute_f1(pred: str, true: str) -> float:
    """
    計算 F1 分數 (前三個字符匹配即算正確)
    
    Args:
        pred: 預測的 ICD 編碼字串,例如 "A010 B021"
        true: 真實的 ICD 編碼字串,例如 "A010 C032"
    
    Returns:
        F1 分數 (0.0 到 1.0)
    """
    # 將字串轉換為 set
    pred = pred.replace('.', '')
    true = true.replace('.', '')
    
    def parse_codes(text):
        if ',' in text:
            return set(code.strip().upper() for code in text.split(',') if code.strip())
        else:
            return set(code.strip().upper() for code in text.split() if code.strip())
    
    pred_set = parse_codes(pred)
    true_set = parse_codes(true)
    
    # 如果都是空集合,視為完全匹配
    if not pred_set and not true_set:
        return 1.0
    
    # 轉換為前三個字符的集合
    def get_prefix_set(codes):
        return set(code[:3] for code in codes if len(code) >= 3)
    
    pred_prefix = get_prefix_set(pred_set)
    true_prefix = get_prefix_set(true_set)
    
    # 計算 TP, FP, FN
    tp = len(pred_prefix & true_prefix)
    fp = len(pred_prefix - true_prefix)
    fn = len(true_prefix - pred_prefix)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    if precision + recall == 0:
        return 0.0
    
    f1 = 2 * precision * recall / (precision + recall)
    return f1



def count_xml(text: str) -> float:
    """
    檢查 XML 標籤格式的完整性
    
    Returns:
        分數 (0.0 到 0.5),分數越高表示格式越完整
    """
    score = 0.0
    
    if text.count("<reasoning>\n") == 1:
        score += 0.125
    if text.count("\n</reasoning>\n") == 1:
        score += 0.125
    if text.count("\n<answer>\n") == 1:
        score += 0.125
        # 懲罰 </answer> 後有多餘內容
        extra_content = len(text.split("\n</answer>\n")[-1])
        score -= extra_content * 0.001
    if text.count("\n</answer>") == 1:
        score += 0.125
        # 懲罰 </answer> 後有多餘內容
        extra_content = len(text.split("\n</answer>")[-1]) - 1
        score -= max(0, extra_content) * 0.001
    
    return max(0.0, score)  # ✅ 確保不會是負數

In [ ]:
START_INDEX = 505     # 起始索引(從0開始)
END_INDEX = 700      # 結束索引(不包含此索引)

# --- 4. 載入 Gemini 模型 ---

genai.configure(api_key=" ")

print(f"✅ 正在設定模型: gemini-2.5-flash ...")
print(f"📊 處理區間: 第 {START_INDEX} 至 第 {END_INDEX-1} 筆 (共 {END_INDEX-START_INDEX} 筆)\n")

# generation_config = {
#     "temperature": 0,
#     "top_p": 1,
#     "top_k": 1,
#     "max_output_tokens": 8192,
# }

# 建立模型實例
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=SYSTEM_PROMPT
)

print("✅ 模型設定完畢。")

# --- 5. 載入並處理資料 ---
print("\n📂 正在載入 note_icd_data.jsonl ...")
chat_style_data = []

try:
    with open("note_icd_data.jsonl", "r", encoding="utf-8") as f:
        all_lines = f.readlines()
        total_records = len(all_lines)
        
        print(f"📊 檔案總共有 {total_records} 筆資料")
        
        # 驗證索引範圍
        if START_INDEX < 0 or END_INDEX > total_records:
            print(f"❌ 錯誤：索引範圍超出資料範圍 (0-{total_records})")
            exit()
        
        if START_INDEX >= END_INDEX:
            print(f"❌ 錯誤：起始索引必須小於結束索引")
            exit()
        
        # 只讀取指定區間的資料
        lines = all_lines[START_INDEX:END_INDEX]
        print(f"✅ 已選取第 {START_INDEX} 至 第 {END_INDEX-1} 筆，共 {len(lines)} 筆\n")

        for line in tqdm(lines, desc="[Gemini] 載入並預處理資料"):
            item = json.loads(line)
            ground_truth_answer = extract_icd_answer(item["answer"])
            
            chat_style_data.append({
                "answer": ground_truth_answer,
                "raw_question": item["question"]
            })

except FileNotFoundError:
    print("❌ 錯誤：找不到 'note_icd_data.jsonl' 檔案。")
    exit()

print(f"✅ 資料載入完畢，共 {len(chat_style_data)} 筆。\n")

✅ 正在設定模型: gemini-2.5-flash ...
📊 處理區間: 第 505 至 第 699 筆 (共 195 筆)

✅ 模型設定完畢。

📂 正在載入 note_icd_data.jsonl ...
📊 檔案總共有 3852 筆資料
✅ 已選取第 505 至 第 699 筆，共 195 筆



[Gemini] 載入並預處理資料: 100%|██████████| 195/195 [00:00<00:00, 57119.16it/s]

✅ 資料載入完畢，共 195 筆。



In [42]:
# --- 6. 執行推論並計算指標 ---
all_predictions = []
all_ground_truths = []
all_f1_scores = []
all_completion_lengths = []

# 生成帶區間標記的 CSV 檔名
csv_filename = f"gemini_flash_results_{START_INDEX}_{END_INDEX}.csv"

# 檢查檔案是否已存在
file_exists = os.path.exists(csv_filename)

with open(csv_filename, 'a', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['index', 'accuracy_reward', 'completion_length', 'ground_truths_ans', 'predictions_ans', 'assistant_reply']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # 只在檔案不存在時寫入表頭
    if not file_exists:
        writer.writeheader()
        print(f"📁 CSV 檔案已建立: {csv_filename}")
    else:
        print(f"📁 CSV 檔案已存在，將追加資料: {csv_filename}")
    
    print(f"📊 開始處理區間 {START_INDEX}-{END_INDEX-1}\n")
    
    # 開始推論
    for idx, item in enumerate(tqdm(chat_style_data, desc="[Gemini] 執行推論與評估"), START_INDEX):
        try:
            question = item["raw_question"]
            gt_answer = item["answer"]
            
            # 建立包含 one-shot 範例的完整對話
            chat = model.start_chat(history=[
                {"role": "user", "parts": [EXAMPLE_QUESTION]},
                {"role": "model", "parts": [f"<reasoning>{EXAMPLE_REASONING}</reasoning><answer>{EXAMPLE_ANSWER}</answer>"]}
            ])
            
            # 發送問題並取得回應
            response = chat.send_message(question)
            assistant_reply = response.text
            
            # 印出完整資訊
            print("\n" + "="*80)
            print(f"📝 樣本 #{idx} (絕對索引)")
            print("="*80)
            print(f"\n🤖 模型完整回答:")
            print("-"*80)
            print(assistant_reply)
            print("-"*80)
            
            # 記錄 ground truth
            all_ground_truths.append(gt_answer)
            
            # 計算 completion length
            completion_length = len(assistant_reply)
            print(f"\n📊 完成的回覆長度 (字元數): {completion_length}")
            all_completion_lengths.append(completion_length)
            
            # 提取預測的 ICD 編碼
            predicted_codes = extract_icd_answer(assistant_reply)
            print(f"🔍 提取的 ICD-10 編碼: {predicted_codes}")
            all_predictions.append(predicted_codes)
            
            print(f"\n🎯 Ground Truth Answer:")
            print(f"   {gt_answer}")
            
            # 計算 F1 分數
            f1_score = compute_f1(predicted_codes, gt_answer)
            print(f"✅ F1 分數: {f1_score:.4f}")
            all_f1_scores.append(f1_score)
            
            # 寫入 CSV (包含絕對索引)
            writer.writerow({
                'index': idx,
                'accuracy_reward': f1_score,    
                'completion_length': completion_length,
                'ground_truths_ans': gt_answer if isinstance(gt_answer, str) else json.dumps(gt_answer), 
                'predictions_ans': json.dumps(predicted_codes) if isinstance(predicted_codes, list) else predicted_codes,
                'assistant_reply': assistant_reply if isinstance(assistant_reply, str) else json.dumps(assistant_reply)
            })
            csv_file.flush()
            
            print(f"💾 結果已儲存至 {csv_filename} (第 {idx} 筆)")
            print("="*80 + "\n")
            
        except Exception as e:
            print(f"\n❌ 處理第 {idx} 筆資料時發生錯誤:")
            print(f"   錯誤訊息: {e}")
            print(f"   跳過此筆資料繼續執行...\n")
            continue

print("\n" + "="*80)
print("🎉 推論完成!")
print("="*80)

# --- 7. 顯示最終統計 ---
if not all_predictions:
    print("⚠️  沒有可評估的結果。")
else:
    print("\n" + "="*50)
    print(f"  Gemini-2.5-Flash 區間統計 [{START_INDEX}-{END_INDEX-1}]")
    print("="*50)
    print(f"✅ 總共處理: {len(all_f1_scores)} 筆資料")
    print(f"📊 平均 F1 分數: {sum(all_f1_scores)/len(all_f1_scores):.4f}")
    print(f"📏 平均回覆長度: {sum(all_completion_lengths)/len(all_completion_lengths):.1f} 字元")
    print(f"💾 結果已完整儲存至: {csv_filename}")
    print("="*50)

📁 CSV 檔案已存在，將追加資料: gemini_flash_results_505_700.csv
📊 開始處理區間 505-699



[Gemini] 執行推論與評估:   1%|          | 1/195 [00:23<1:17:13, 23.88s/it]


📝 樣本 #505 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **Chief Complaint: shortness of breath** - 這是患者就診的主要原因，應編碼為 R06.02 (Shortness of breath, unspecified)。
2.  **History of Present Illness: hx of HLD (Hyperlipidemia)** - 患者既往病史中的高血脂，應編碼為 E78.5 (Unspecified hyperlipidemia)。
3.  **History of Present Illness: hypothyroidism** - 患者既往病史中的甲狀腺功能低下，應編碼為 E03.9 (Unspecified hypothyroidism)。
4.  **History of Present Illness: subglottic stenosis** - 患者的核心診斷，也是本次就診的主要原因，應編碼為 J38.6 (Stenosis of larynx)。
5.  **History of Present Illness: increased worsening shortness of breath with inspiration x 4 weeks, dyspnea with exertion and stridor** - 這些是亞聲門狹窄的表現， dyspnea (呼吸困難) 已經被 shortness of breath 涵蓋 (R06.02)。Stridor (喘鳴) 是一個額外的重要症狀，應編碼為 R06.1 (Stridor)。
6.  **History of Present Illness: Patient endorses phlegm which is white and stems from her throat** - 異常痰液是一種症狀，應編碼為 R09.3 (Abnormal sputum)。
7.  **Allergies: Morphine** - 對特定

[Gemini] 執行推論與評估:   1%|          | 2/195 [01:08<1:55:20, 35.86s/it]


📝 樣本 #506 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **Fevers, purulent drainage (Chief Complaint):** 這些是患者入院的主要症狀，提示存在感染。
    *   **R50.9** (Fever, unspecified)
2.  **Sternal wound infection with purulent drainage, involving bone:** 病史中明確提到"new purulent JP drainage that grew Proteus/MRSA from right and Pseudomonas from left JP"以及"Debridement of Sternal wound to and including bone"。這表明是一個深部手術部位感染，並已發展為骨髓炎。
    *   **T81.4XXA** (Infection following a procedure, initial encounter) - 這是手術後感染的通用碼。
    *   **M86.80** (Other osteomyelitis, unspecified site) - 由於胸骨骨髓炎沒有更具體的ICD-10-CM編碼，此碼適用於明確記載為骨髓炎但無更細分部位碼的情況。
3.  **多重細菌感染 (MRSA, Pseudomonas, Proteus):** 檢測結果顯示有三種細菌。
    *   **B95.61** (Methicillin resistant Staphylococcus aureus infection as the cause of diseases classified elsewhere)
    *   **B96.5** (Pseudomonas as the cause of diseases classified elsewhere)
    *   **B96.2** (Proteus as the cause of diseases c

...


📝 樣本 #697 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **Fevers, back pain (Chief Complaint):** These are symptoms that will likely be covered by more specific diagnoses. However, persistent fever could be R50.9 and back pain could be M54.9 or more specific.
2.  **Paraplegia with chronic foley:** Paraplegia is a significant PMH. This maps to G82.20 (Paraplegia, unspecified). The chronic foley indicates an indwelling catheter, coded as Z96.0 (Presence of indwelling urinary catheter).
3.  **Chronic sacral ulcer:** This is a pressure ulcer. Since no stage is specified, it is coded as L89.119 (Pressure ulcer of sacral region, unspecified stage).
4.  **DM2:** Type 2 Diabetes Mellitus. No complications are mentioned, so E11.9 (Type 2 diabetes mellitus without complications) is appropriate.
5.  **Seizure d/o:** Seizure disorder. Without further specification, G40.909 (Epilepsy, unspecified, not intractable, without s

[Gemini] 執行推論與評估:  99%|█████████▉| 194/195 [58:10<00:20, 20.79s/it]


📝 樣本 #698 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **Dementia:** 病史中有「history of dementia」。這是一個重要的共病症。對應編碼為 G30.9 (Dementia, unspecified)。
2.  **Abdominal pain:** 患者主訴為「abdominal pain」，且病史中也提及「diffuse abd pain for the past 2 days」和「abdominal pain since the beginning of [month]」。對應編碼為 R10.9 (Unspecified abdominal pain)。
3.  **Suicidal ideation:** 患者表示「she has thoughts of self-harm」。這是一個需要立即處理的心理健康問題。對應編碼為 R45.851 (Suicidal ideation)。
4.  **Acute pyelonephritis:** CT 報告指出「Hypodense regions are seen in the right kidney with possibly striated nephrogram, concerning for early pyelonephritis」。尿液分析顯示「51 WBC, 113 RBC, Lg leuks, Moderate blood, neg Nitrites, protein 30, few bacteria」。這些都強烈指向急性腎盂腎炎。對應編碼為 N10 (Acute pyelonephritis)。
5.  **Noninfective gastroenteritis and colitis:** CT 報告提及「mild wall thickening of the descending colon with associated fat stranding, possibly mild colitis」。對應編碼為 K52.9 (Noninfective gast

[Gemini] 執行推論與評估: 100%|██████████| 195/195 [58:39<00:00, 18.05s/it]


📝 樣本 #699 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **Atrial Fibrillation (Afib)**: 患者有心房顫動病史，使用Xarelto，且曾接受燒灼術 (s/p ablation)。此應編碼為 I48.91 (Unspecified atrial fibrillation)。
2.  **COPD**: 患者有輕度COPD，肺功能指標FEV1/FVC 60%，80% of predicted。此應編碼為 J44.9 (Chronic obstructive pulmonary disease, unspecified)。
3.  **Diastolic CHF (Heart failure with preserved ejection fraction - HFpEF)**: 患者有舒張性心衰竭病史，EF 50%，伴有3+二尖瓣反流 (MR)，並有慢性進展性呼吸困難、端坐呼吸等症狀。此應編碼為 I50.33 (Chronic diastolic heart failure)。
4.  **DM (Diabetes Mellitus)**: 患者有糖尿病病史，A1c 6.3%。儘管A1c在此值可能被視為糖尿病前期，但病歷明確寫為"DM"，故應編碼為 E11.9 (Type 2 diabetes mellitus without complications)。
5.  **Tracheobronchomalacia**: 患者有輕度氣管支氣管軟化症。此應編碼為 J98.01 (Tracheobronchomalacia)。
6.  **HTN (Hypertension)**: 患者有高血壓病史。此應編碼為 I10 (Essential (primary) hypertension)。
7.  **HLD (Hyperlipidemia)**: 患者有高血脂病史。此應編碼為 E78.5 (Hyperlipidemia, unspecified)。
8.  **CKD (Chronic Kidney Disease)**: 患者有慢性腎病病史

In [43]:
import re

def extract_icd_answer(text: str) -> str:
    """
    從模型的完整回覆或標準答案中提取 ICD-10 編碼。
    優先從 <answer> 標籤中提取,若無則從整個文本中提取。
    """
    # 使用正則表達式搜尋 <answer> 標籤，re.DOTALL 讓 . 可匹配換行符，re.IGNORECASE 忽略大小寫
    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL | re.IGNORECASE)
    # 如果找到 <answer> 標籤，則只在標籤內容中搜尋 ICD 編碼；否則在整個文本中搜尋
    search_text = answer_match.group(1) if answer_match else text
    
    # 匹配格式：字母 + 1-3位數字 + 可選的小數點和1-2位數字
    codes_list = re.findall(r'\b([A-Z]\d{1,6}(?:\.\d{1,2})?)\b', search_text, re.IGNORECASE)
    
    # 移除小數點、去重、排序、轉大寫，並過濾掉長度<=2的編碼
    cleaned_codes = sorted(set(
        code.replace('.', '').upper() 
        for code in codes_list 
        if len(code.replace('.', '')) > 2
    ))
    return " ".join(cleaned_codes)


# 測試字串
test_text = """D550,O09212,O26872,O30042,O99012,Z3A18。"""

# 執行測試
result = extract_icd_answer(test_text)
print(f"提取結果: {result}")
print(f"預期結果: D550 O09212 O26872 O30042 O99012 Z3A18")
print(f"測試通過: {result == 'R074 R109'}")

提取結果: D550 O09212 O26872 O30042 O99012
預期結果: D550 O09212 O26872 O30042 O99012 Z3A18
測試通過: False
